<a href="https://colab.research.google.com/github/isakdiaz/treeseg-firebase-train/blob/main/treeseg_coreml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CoreML Conversion from Tensorflow Model
Notebook is for loading DeepLabV3 tree segmentation model from google drive and doing a CoreML conversion so you can generate an mlmodel file that runs on iOS devices. Run treeseg_train.ipynb first if you still do not have a model.

In [1]:
%tensorflow_version 2.x
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import json
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import json
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm


from tensorflow.keras.callbacks import ModelCheckpoint
print(f"Running tensorflow version {tf.__version__}")

Running tensorflow version 2.7.0


## Load Cloud Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install CoreML Tools
(Requires Python 3.5 +)

In [3]:
!pip install coremltools
import coremltools as ct

     |████████████████████████████████| 1.6 MB 9.8 MB/s 


# Load Model File
Root directory of my google drive has saved_models folder which contains models. If you ran treeseg_train.ipynb
 your model folder should be in the same folder.

In [4]:
# Check if a model exists
GDRIVE_FOLDER = "/content/drive/MyDrive"
MODEL_FOLDER = "saved_models/treeseg"

!ls -all -h {os.path.join(GDRIVE_FOLDER, MODEL_FOLDER)}

total 1.1G
-rw------- 1 root root 206M Aug 29 16:09 treeseg2021-08-29.h5
-rw------- 1 root root 206M Sep  1 04:31 treeseg_2021-09-01.h5
-rw------- 1 root root 206M Sep  2 17:02 treeseg_2021-09-02.h5
-rw------- 1 root root 206M Sep 16 03:50 treeseg_2021-09-16.h5
-rw------- 1 root root 206M Jan 15 22:10 treeseg_2022-01-15.h5


In [5]:
# Pick a model and replace model filename parameter
MODEL_FILENAME = "treeseg_2022-01-15.h5"

model_filepath = os.path.join(GDRIVE_FOLDER, MODEL_FOLDER, MODEL_FILENAME)

## Load custom loss functions

In [6]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Load Model

In [7]:
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
  model = tf.keras.models.load_model(model_filepath)

print(f"Model loaded from path {model_filepath}")

Model loaded from path /content/drive/MyDrive/saved_models/treeseg/treeseg_2022-01-15.h5


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

# Convert Keras model to CoreML Model


In [9]:
# image_input = ct.ImageType(color_layout="RGB", scale=1/127.0, bias=[-1,-1,-1])
image_input = ct.ImageType(color_layout="RGB", scale=1/255.0, bias=[0,0,0])
# classifier_config = ct.ClassifierConfig(labels)

# Set input as ImageType so CoreML can automatically resize it using Vision framework
coreml_model = ct.convert(model, inputs=[image_input])

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 674/674 [00:03<00:00, 221.16 ops/s]


## Get Model input specs
There should be a 1/255.0 scaler, The model input is between 0 to 1

In [10]:
# # Define Spec Function
from coremltools.models.neural_network.builder import _get_nn_spec as get_nn

## Get Spec and check preprocessing
spec = coreml_model.get_spec()
nn = get_nn(spec)
print(nn.preprocessing)

[featureName: "input_1"
scaler {
  channelScale: 0.003921568859368563
}
]


# Save CoreML Model

## Create model folder

In [11]:
model_name = MODEL_FILENAME.split("/")[-1]
folder_name = GDRIVE_FOLDER + "/CoreML"
!mkdir -p {folder_name}

## Save model to google drive

In [12]:
coreml_file_path = "{0}/{1}.mlmodel".format(folder_name, model_name.split(".")[0])
coreml_model.save(coreml_file_path)
print("Core ML model {} saved in {}".format(model_name.split(".")[0], folder_name))

Core ML model treeseg_2022-01-15 saved in /content/drive/MyDrive/CoreML
